In [1]:
from os import listdir
from os.path import isfile, join
from skimage import io
from skimage.segmentation import felzenszwalb, slic, quickshift, join_segmentations
from skimage.measure import label, regionprops
import numpy as np
import scipy as sp
from datetime import datetime
from utils import mahalanobis
import warnings
warnings.filterwarnings("ignore")

In [2]:
def get_neighbors(label_img):
    #print(label_img.shape)
    props = regionprops(label_img)
    vizinhos_original = []
    vizinhos_original.append([])
    len_props=len(props)
    for i in range(0,len_props):
        #if (i)%int(len_props/25) == 0:
        #    print(i)
        vizinhos_original.append([])
        for j in props[i].coords:
            if j[0]-1>=0:
                if props[i].label!=label_img[j[0]-1,j[1]]:
                    temp=label_img[j[0]-1,j[1]]
                    vizinhos_original[props[i].label].append(temp)
            if j[1]-1>=0:
                if props[i].label!=label_img[j[0],j[1]-1]:
                    vizinhos_original[props[i].label].append(label_img[j[0],j[1]-1])
            if j[0]+1<label_img.shape[0]:
                if props[i].label!=label_img[j[0]+1,j[1]]:
                    vizinhos_original[props[i].label].append(label_img[j[0]+1,j[1]])
            if j[1]+1<label_img.shape[1]:
                if props[i].label!=label_img[j[0],j[1]+1]:
                    vizinhos_original[props[i].label].append(label_img[j[0],j[1]+1])
        vizinhos_original[props[i].label] = list(np.unique(vizinhos_original[props[i].label]))
    #print(len_props)
    return vizinhos_original

In [3]:
def FuSS(label_img, dataset, filename, min_number_pixels=50, distance='mean'):
    print(dataset,'-',filename,'|','gerando labels unicos')
    label_img = label(label_img, connectivity=1)
    print(dataset,'-',filename,'|','gerando vizinhos unicos')
    vizinhos = get_neighbors(label_img)
    print(dataset,'-',filename,'|','gerando segmentos de tamanho minimo',min_number_pixels)
    t1=datetime.now()
    counter=0
    k=np.min(label_img)
    label_img_max=np.max(label_img)
    to_print = int(label_img_max/1000)
    if to_print<250:
        to_print=250
    while (k <= label_img_max):
        t2=datetime.now()
        i=k

        label_img_i_len = len(label_img[label_img==i])
        if (label_img_i_len>=min_number_pixels) or (label_img_i_len==0):
            k+=1
            continue

        minDistance = 9999999999999999999999
        minDistanceSeg = -1

        for v in vizinhos[i]:
            if v==i:
                continue
            if minDistanceSeg == -1:
                minDistanceSeg = v
            m=99999999999999999999999
            len_v = len(label_img[label_img==v])
            if len_v==0:
                continue
            if len_v>label_img_i_len and len_v>2:
                if distance=='mean':
                    m = mahalanobis(np.mean(img[label_img==i], axis=0),img[label_img==v])
                else:
                    m = mahalanobis(np.median(img[label_img==i], axis=0),img[label_img==v])
                
            elif label_img_i_len>2: 
                if distance=='mean':
                    m = mahalanobis(np.mean(img[label_img==v], axis=0),img[label_img==i])
                else:
                    m = mahalanobis(np.median(img[label_img==v], axis=0),img[label_img==i])
            if m<minDistance:
                minDistance = m
                minDistanceSeg = v

        finalsegment = -1
        removedsegment = -1
        if minDistanceSeg<i:
            finalsegment = minDistanceSeg
            removedsegment = i
        else:
            finalsegment = i
            removedsegment = minDistanceSeg

        label_img[label_img==removedsegment] = finalsegment

        finalsegment_len = len(label_img[label_img==finalsegment])

        if finalsegment<=k and finalsegment_len<min_number_pixels:
            k=finalsegment
        else:
            k+=1

        vizinhos[finalsegment] = [*vizinhos[finalsegment],*vizinhos[removedsegment]]
        vizinhos[finalsegment] = list(np.unique(vizinhos[finalsegment]))
        temp = vizinhos[removedsegment].copy()
        for j in temp:
            try:
                vizinhos[j].remove(removedsegment)
            except:
                print('ERRO removendo:',removedsegment)
            vizinhos[j].append(finalsegment)

            vizinhos[j] = list(np.unique(vizinhos[j]))

        counter+=1
        if counter%to_print==0:
            #print(label_img_max,'|',k,'-',i,'-',finalsegment, '-', removedsegment,"|", np.max(label_img), '|', datetime.now()-t1, datetime.now()-t2, "-", counter, '-', (datetime.now()-t1)/counter," | ",'final segment len: ',finalsegment_len)#, '|', len(vizinhos[finalsegment-1]))
            print(dataset,'-',filename,'|',label_img_max,'|',k,'|', datetime.now()-t1, datetime.now()-t2, "-", counter, '-', (datetime.now()-t1)/counter)
        if k > label_img_max:
            label_img = label(label_img, connectivity=1)
            unique_labels = np.unique(label_img, return_counts=True)
            smaller_than_min = len(unique_labels[1][unique_labels[1]<min_number_pixels])
            print(unique_labels[0][unique_labels[1]<min_number_pixels])
            print(unique_labels[1][unique_labels[1]<min_number_pixels])
            if smaller_than_min>0:
                print("reprocessing: ",smaller_than_min)
                k=np.min(label_img)
                label_img_max=np.max(label_img)
                vizinhos = get_neighbors(label_img)            
            else:
                break

    label_img = label(label_img, connectivity=1)
    return label_img

In [4]:
datasets=['Vaihingen']
distances=['mean']#,'median']
min_pixels=[50]
datasets_path = '../datasets'

- the dataset folder must contain a folder named after the dataset itself
- The dataset folder must contain the subfolder 'Test' with 'JPEGImages' containing the images to segment.
- The 'Test' folder must also contain a subfolder 'superpixels' to save the segmented images.

All tested superpixel segmentations are shown in the next cell.

In [ ]:
for dataset in datasets:
    for distance in distances:
        for min_pixel in min_pixels:
            
            superpixels_path = join(datasets_path,dataset,'Test','superpixels')

            images_path=join(datasets_path,dataset,'Test','JPEGImages') 
            listimages = listdir(images_path)

            min_number_pixels=int(min_pixel)
            
            for i in listimages:
                save_image_name_path = join(superpixels_path,i)
                img = io.imread(join(images_path,i))
                
                print("segmenting image:",i)

                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/2000), compactness=5, sigma=1, start_label=1),felzenszwalb(img, scale=200, sigma=0.7, min_size=200))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_slic01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1500), compactness=5, sigma=1, start_label=1),felzenszwalb(img, scale=100, sigma=0.7, min_size=150))
                segments = FuSS(segments, dataset, i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_slic02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1000), compactness=5, sigma=1, start_label=1),felzenszwalb(img, scale=100, sigma=0.7, min_size=150))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_slic03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1),felzenszwalb(img, scale=50, sigma=0.7, min_size=100))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_slic04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                print('Felzenszwalb-SLIC done',i)

                segments = felzenszwalb(img, scale=50, sigma=0.5, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=100, sigma=0.5, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=200, sigma=0.5, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=400, sigma=0.5, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=50, sigma=0.5, min_size=100)
                np.savez(save_image_name_path.replace('.tif','_segments_fz05_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=100, sigma=0.5, min_size=100)
                np.savez(save_image_name_path.replace('.tif','_segments_fz06_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=50, sigma=0.7, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz07_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=100, sigma=0.7, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz08_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=200, sigma=0.7, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz09_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=400, sigma=0.7, min_size=50)
                np.savez(save_image_name_path.replace('.tif','_segments_fz10_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=50, sigma=0.7, min_size=100)
                np.savez(save_image_name_path.replace('.tif','_segments_fz11_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = felzenszwalb(img, scale=100, sigma=0.7, min_size=100)
                np.savez(save_image_name_path.replace('.tif','_segments_fz12_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                print('Felzenszwalb done',i)

                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/1400), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/700), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/350), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/230), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic05_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = slic(img, n_segments=int(img.shape[0]*img.shape[1]/170), compactness=5, sigma=1, start_label=1)
                np.savez(save_image_name_path.replace('.tif','_segments_slic06_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                print('SLIC done')

                segments = quickshift(img, kernel_size=2, max_dist=50, ratio=0.5)
                np.savez(save_image_name_path.replace('.tif','_segments_quick01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = quickshift(img, kernel_size=3, max_dist=50, ratio=0.5)
                np.savez(save_image_name_path.replace('.tif','_segments_quick02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = quickshift(img, kernel_size=4, max_dist=50, ratio=0.5)
                np.savez(save_image_name_path.replace('.tif','_segments_quick03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = quickshift(img, kernel_size=5, max_dist=50, ratio=0.5)
                np.savez(save_image_name_path.replace('.tif','_segments_quick04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                print('Quick done',i)

                segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=2, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_quick01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=3, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_quick02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=4, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_quick03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(felzenszwalb(img, scale=200, sigma=0.7, min_size=200), quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_fz_quick04_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                print('Felzenszwalb-Quick done',i)

                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/2000), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_quick_slic01_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1500), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_quick_slic02_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/1000), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_quick_slic03_'+distance+'_'+str(min_number_pixels)+'.npz'),segments)
                segments = join_segmentations(slic(img, n_segments=int(img.shape[0]*img.shape[1]/500), compactness=5, sigma=1, start_label=1),quickshift(img, kernel_size=5, max_dist=50, ratio=0.5))
                segments = FuSS(segments, dataset,i, min_number_pixels=min_number_pixels, distance=distance)
                np.savez(save_image_name_path.replace('.tif','_segments_quick_slic04_'+distance+'_'+str(min_number_pixels)+'.npz'), segments)

                print('Quick-SLIC done',i)

segmenting image: top_mosaic_09cm_area11.tif
Vaihingen - top_mosaic_09cm_area11.tif | gerando labels unicos
Vaihingen - top_mosaic_09cm_area11.tif | gerando vizinhos unicos
Vaihingen - top_mosaic_09cm_area11.tif | gerando segmentos de tamanho minimo 200
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 327 | 0:00:47.805964 0:00:00.108722 - 250 - 0:00:00.191224
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 608 | 0:01:21.424322 0:00:00.119807 - 500 - 0:00:00.162849
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 875 | 0:02:11.848470 0:00:00.200170 - 750 - 0:00:00.175798
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 1148 | 0:02:59.156638 0:00:00.154383 - 1000 - 0:00:00.179157
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 1416 | 0:03:42.749227 0:00:00.117789 - 1250 - 0:00:00.178199
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 1684 | 0:04:26.451151 0:00:00.163739 - 1500 - 0:00:00.177634
Vaihingen - top_mosaic_09cm_area11.tif | 49302 | 1958 | 0:05:17.126233 0:00:00.151096 - 1750